In [1]:
# !pip install geopy

In [2]:
# sudo systemctl start postgresql
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

In [3]:
# import re
# import json
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

tqdm.pandas()
pd.options.display.max_colwidth = None 

---

In [4]:
# !ls data/location

In [5]:
# !ls data

In [6]:
ts = '2022-09-02_13-04'
location_path = 'data/location/avito_2022-09-01_14-07_location.pkl' #  старый список адресов
data_path = f'data/avito_{ts}.xlsx'

---

In [7]:
# location = ( # список адресов из новых данных
#     pd.read_excel(data_path)[['adr']]
#     .drop_duplicates()
#     .merge(pd.read_pickle(location_path),on=['adr'],how='outer')
# )
# print(len(location))
# # location.sample(2)

# # список адресов без метки
# location_ = location[ location['coo'].isnull() ][['adr']].reset_index(drop=True)
# location_

---

In [8]:
location_ = pd.read_excel(data_path)[['adr']].drop_duplicates().reset_index(drop=True)
location_

,adr
0,"улица Генерала Крейзера, д. 8, корп. 5"
1,"проспект Победы, 29В"
2,"улица Военных Строителей, стр. 6.3"
3,"аллея Защитников 35-й Батареи, д. 11а"
4,"улица Генерала Мельника, 11А/1"
...,...
1579,"улица Павла Корчагина, 5"
1580,"село Вилино, улица Чапаева, 10"
1581,"СТ Рубин, 58"
1582,"Кача, улица Авиаторов, 34"


In [9]:
import re
location_['adr_'] = (
        location_['adr']
        .apply(lambda s: re.sub(r'\b[Сс]евастополь\b','',s))
        .apply(lambda s: re.sub(r'^','севастополь, ',s))
        .apply(lambda s: re.sub(r'севастополь.*село\b','крым, село ',s))
        .apply(lambda s: re.sub(r'\bмикрорайон\b',' ',s))
        .apply(lambda s: re.sub(r'\bкотт?еджный пос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r'\bпос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r',\s*,',', ',s))
        .apply(lambda s: re.sub(r' +',' ',s))
    )

In [11]:
# location_[['adr_']].sort_values(by='adr_').to_csv('tmp/adr.csv',index=False)

In [12]:
# наб.
# пл. 
# пос. 
# пр-д 
# пр.
# садоводческое некоммерческое товарищество
# садоводческое товарищество

---

In [13]:
# запускаем геокодер

# sudo systemctl start postgresql
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

geolocator = Nominatim(user_agent='http://127.0.0.1:8088')

# adr = 'севастополь острякова 57'
# loc = geolocator.geocode(adr)
# (loc.latitude, loc.longitude)

In [14]:
def get_coo(adr, city_prefix='Севастополь', geolocator=geolocator):
    # добавляем название города к адресу 
    if len(city_prefix)>0: adr_ =  city_prefix + ', ' + adr
    
    # пробуем определить координаты
    adr_ = adr_.split(',')
    loc = geolocator.geocode(adr_)
    
    truncated = False
    while (loc is None) and ( len(adr_) > 1 ): # если не нашел...
        adr_ = adr_[:-1] # выкидываем часть адреса 
        loc = geolocator.geocode( adr_) # ... и пробуем ещё раз
        truncated = True
        
    return (
        {'coo':None,'truncated':None} 
        if (loc is None) 
        else {'coo':(loc.latitude, loc.longitude),'truncated':truncated} 
    )

In [15]:
%%time

# ищем метку по адресу

location_['coo'] = location_['adr_'].progress_apply(get_coo)
location_['is_adr_truncated'] = location_['coo'].str['truncated']
location_['coo'] = location_['coo'].str['coo']

location_.sample(4)

  0%|          | 0/1584 [00:00<?, ?it/s]

CPU times: user 15.7 s, sys: 889 ms, total: 16.6 s
Wall time: 18min 44s


,adr,adr_,coo,is_adr_truncated
28,"улица Будищева, 44","севастополь, улица Будищева, 44","(44.6004897, 33.53772710363263)",False
1040,"Флагманская улица , 8","севастополь, Флагманская улица , 8","(44.571954399999996, 33.53202190000001)",False
668,"улица Челюскинцев, 49","севастополь, улица Челюскинцев, 49","(44.630622, 33.5404022)",False
317,"улица Курчатова, 15","севастополь, улица Курчатова, 15","(44.6280141, 33.5699902)",False


In [19]:
# список адресов для которых не удалось найти метку 
location_[ location_['coo'].isnull() ]

,adr,adr_,coo,is_adr_truncated


In [21]:
location_[ location_['is_adr_truncated'] ]

,adr,adr_,coo,is_adr_truncated
0,"улица Генерала Крейзера, д. 8, корп. 5","севастополь, улица Генерала Крейзера, д. 8, корп. 5","(44.6044898, 33.5120778)",True
2,"улица Военных Строителей, стр. 6.3","севастополь, улица Военных Строителей, стр. 6.3","(44.5679458, 33.4055903)",True
3,"аллея Защитников 35-й Батареи, д. 11а","севастополь, аллея Защитников 35-й Батареи, д. 11а","(44.6054434, 33.5220842)",True
4,"улица Генерала Мельника, 11А/1","севастополь, улица Генерала Мельника, 11А/1","(44.5767843, 33.5652905)",True
5,"шоссе Лабораторное, д. 33, секц. 6","севастополь, шоссе Лабораторное, д. 33, секц. 6","(44.5746091, 33.5809925)",True
...,...,...,...,...
1574,"улица Ивана Голубца, 104","севастополь, улица Ивана Голубца, 104","(44.5909497, 33.5118155)",True
1577,"проспект Античный, д. 26, корп. 5","севастополь, проспект Античный, д. 26, корп. 5","(44.5941174, 33.43597505148334)",True
1579,"улица Павла Корчагина, 5","севастополь, улица Павла Корчагина, 5","(44.5833504, 33.4336844)",True
1580,"село Вилино, улица Чапаева, 10","крым, село Вилино, улица Чапаева, 10","(44.5736594, 33.400570902016185)",True


In [ ]:
# # собираем локации в один список
# location_ = pd.concat([
#     location[ ~location['coo'].isnull() ],
#     location_[ ~location_['coo'].isnull() ]
# ]).drop_duplicates().reset_index(drop=True)

In [20]:
# location.to_excel(f'data/avito_{ts}_location.xlsx', index=False)
location_.to_pickle(f'data/location/avito_{ts}_location.pkl')

----

In [ ]:
# df = pd.read_excel(data_path).merge(location_,on=['adr'],how='left')
# df

In [ ]:
# df[df['coo'].isnull()][['adr']].drop_duplicates()#.sort_values(by='adr')#.to_csv('tmp/adr.csv',index=False)

In [ ]:
# df[ df['adr'].str.match(r'.*городского.*') ][['adr','coo']].drop_duplicates()

In [ ]:
# import re
# s= 'ш. Авиаторов'
# s= 'Авиаторов ш.'

# # пос. городского типа

# re.sub(r'\bш\.','шоссе ',s)

In [ ]:
# пер. -> перeулок
# ш. Ла -> шоссе
# б-р -> бульвар
# c. - село


In [ ]:
# adr = 'севастополь, 4-й Хомутов перeулок'
# adr = 'севастополь, Античный бульвар, 24'
# adr = 'севастополь,Английский бульвар, 22'
# adr = 'севастополь,ул. Генерала Лебедя'
# adr = 'севастополь, проспект Юрия Гагарина, 31'
# adr = 'крым, село Вилино'

#adr = 'севастополь, ул. Защитников Батареи'
# adr= 'севастополь, СТ Ветеран'# , 4-я Ежевичная ул., 153'

# adr = 'севастополь,  Вилларис дель Мар'
# adr = 'севастополь, Гагаринский муниципальный округ, микрорайон Казачья Бухта, Рубежная улица, 7'
# adr = 'севастополь,  Любимовка, Южногородская улица' # , 36 к7'

# loc = geolocator.geocode(adr)
#(loc.latitude, loc.longitude)
# loc


# Location(ТСН СНТ Ветеран, Гагаринский округ, Гагаринский район, Севастополь, Україна, (44.544144, 33.443218046188065, 0.0))

----

In [ ]:
# geolocator.geocode('Севастополь, Аэродромное шоссе, 2 к34')
# geolocator.geocode('Севастополь, проспект Генерала Острякова, 244 к7')
# geolocator.geocode('Севастополь, ул. Астана Кесаева,')
# geolocator.geocode('Севастополь, ул. Николая Музыки')
# geolocator.geocode('Севастополь, ул. Коли Пищенко, 20А'.split(',')[:-1])
# geolocator.geocode('Севастополь, с.Тыловое, Севастопольская ул., 6')
# geolocator.geocode('Севастополь, с.Тыловое')
# 'Севастополь, ул. Коли Пищенко, 20А'.split(',')[:-1]
# geolocator.geocode('Севастополь, садоводческое некоммерческое товарищество Бриз')
# geolocator.geocode('с. Песчаное, ')

In [ ]:
# geolocator.geocode?

In [ ]:
# import re
# location['adr'].apply(lambda s: re.sub(r'(\d)(к\d)',r'\1 \2',s))

In [ ]:
# geolocator.geocode('Севастополь садоводческое товарищество Кача, 1')

In [ ]:
# !pip install folium

In [ ]:
# import folium

# m = folium.Map(location=coo,zoom_start=23)
# folium.Marker(coo,popup=adr,tooltip=adr).add_to(m)
# m

---

In [ ]:
# !pip install osmnx

In [ ]:
# import osmnx as ox

# center_point = (40.4168, -3.7038)
# G = ox.graph_from_point(center_point, dist=15000, retain_all=True, simplify = True, network_type='all')

---

In [ ]:
# !pip install nominatim

In [ ]:
# from nominatim import Nominatim
# #from nominatim import NominatimReverse

# nom = Nominatim('http://127.0.0.1:8088')

In [ ]:
# nom.query(u'москва', acceptlanguage='RU')
# # nom.query(u'moscow')

In [ ]:
# u'москва'.encode('utf-8').strip()

In [ ]:
# http://127.0.0.1:8088/search.php?q=севастополь

---

In [ ]:
# from ipyleaflet import AwesomeIcon

# # m = Map(center=(55.718148, 37.555493), zoom=14)

In [ ]:
# location['adr_'] = 

# location['coo'] = (
#     (
#     'Севастополь, '
#      +location['adr']
#      .apply(lambda s: re.sub(r'(\d)(к\d)',r'\1 \2',s))
#      .str.replace(' ш.,',' шоссе,')
#      .str.replace('пр-т','проспект')
#     )
#     location['adr_'] 
#     .progress_apply(geolocator.geocode )
#     .apply(lambda x: x if x is None else (x.latitude, x.longitude))
# )

# # ('город Севастополь, '+location['adr']).progress_apply( geolocator.geocode )
# location

# location['coo'] = (
#     ( 'Севастополь, ' + location['adr'] )
#     .progress_apply(geolocator.geocode )
#     .apply(lambda x: x if x is None else (x.latitude, x.longitude))
# )

# ('город Севастополь, '+location['adr']).progress_apply( geolocator.geocode )